<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/multiverse_simulation_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# filename: multiverse_simulation.py

import os
import argparse
import json
import random
from typing import Tuple, Optional

import numpy as np

# Keep TF quieter and nudge determinism before importing tensorflow
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
os.environ.setdefault("TF_DETERMINISTIC_OPS", "1")

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# -----------------------
# Utilities
# -----------------------

def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


def str2bool(v: str) -> bool:
    if isinstance(v, bool):
        return v
    v = v.strip().lower()
    if v in ("yes", "true", "t", "1", "y"):
        return True
    if v in ("no", "false", "f", "0", "n"):
        return False
    raise argparse.ArgumentTypeError(f"Boolean value expected, got: {v}")


def ensure_outdir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


# -----------------------
# Data generation
# -----------------------

def generate_multiverse_data(
    samples: int,
    input_dim: int,
    num_classes: int,
    noise: float = 0.2,
    random_state: int = 42,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Generates learnable synthetic classification data:
    - Features X in [0, 1]^input_dim
    - Class logits via linear separators per class (+ Gaussian noise)
    - Labels are argmax over those logits
    """
    rng = np.random.default_rng(random_state)
    X = rng.random((samples, input_dim)).astype(np.float32)

    # Class-specific linear template
    W = rng.normal(0, 1, size=(input_dim, num_classes)).astype(np.float32)
    b = rng.normal(0, 0.5, size=(num_classes,)).astype(np.float32)

    logits = X @ W + b  # shape: (N, C)
    if noise > 0:
        logits += rng.normal(0, noise, size=logits.shape).astype(np.float32)

    y = np.argmax(logits, axis=1).astype(np.int32)  # shape: (N,)
    return X, y


def make_datasets(
    X: np.ndarray,
    y: np.ndarray,
    val_ratio: float,
    batch_size: int,
    seed: int = 42,
):
    n = X.shape[0]
    idx = np.arange(n)
    rng = np.random.default_rng(seed)
    rng.shuffle(idx)

    n_val = max(1, int(val_ratio * n))
    val_idx = idx[:n_val]
    train_idx = idx[n_val:]

    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]

    train_ds = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .shuffle(buffer_size=X_train.shape[0], seed=seed, reshuffle_each_iteration=True)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return train_ds, val_ds, (X_train, y_train, X_val, y_val)


# -----------------------
# Model
# -----------------------

class MultiverseSimulation(Model):
    def __init__(
        self,
        input_dim: int,
        num_classes: int,
        hidden1: int = 128,
        hidden2: int = 128,
        activation: str = "relu",
        dropout: float = 0.0,
    ):
        super().__init__()
        self.d1 = Dense(hidden1, activation=activation)
        self.drop1 = Dropout(dropout) if dropout and dropout > 0 else None
        self.d2 = Dense(hidden2, activation=activation)
        self.drop2 = Dropout(dropout) if dropout and dropout > 0 else None
        self.out = Dense(num_classes, activation="softmax")
        # Build weights by calling once with a known input shape
        self._input_dim = input_dim

    def build(self, input_shape):
        # Explicitly build layers with known input shapes for saved model signatures
        _ = self.call(tf.zeros((1, self._input_dim), dtype=tf.float32))
        super().build(input_shape)

    def call(self, inputs, training: Optional[bool] = None):
        x = self.d1(inputs)
        if self.drop1 is not None:
            x = self.drop1(x, training=training)
        x = self.d2(x)
        if self.drop2 is not None:
            x = self.drop2(x, training=training)
        return self.out(x)


def build_model(
    input_dim: int,
    num_classes: int,
    hidden1: int,
    hidden2: int,
    activation: str,
    dropout: float,
    learning_rate: float,
) -> tf.keras.Model:
    model = MultiverseSimulation(
        input_dim=input_dim,
        num_classes=num_classes,
        hidden1=hidden1,
        hidden2=hidden2,
        activation=activation,
        dropout=dropout,
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


# -----------------------
# Artifacts
# -----------------------

def save_artifacts(
    outdir: str,
    model: tf.keras.Model,
    config: dict,
    history: dict,
    metrics: dict,
    save_model_flag: bool,
):
    ensure_outdir(outdir)
    if save_model_flag:
        final_path = os.path.join(outdir, "model.keras")
        model.save(final_path)
    with open(os.path.join(outdir, "run_summary.json"), "w") as f:
        json.dump({"config": config, "history": history, "metrics": metrics}, f, indent=2)


# -----------------------
# CLI
# -----------------------

def parse_args():
    parser = argparse.ArgumentParser(
        description="Train a Keras multiverse classifier on synthetic, learnable data (Jupyter/Colab-safe).",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )
    # Data
    parser.add_argument("--samples", type=int, default=2000, help="Number of samples to generate.")
    parser.add_argument("--input_dim", type=int, default=5, help="Number of input features.")
    parser.add_argument("--num_classes", type=int, default=10, help="Number of classes.")
    parser.add_argument("--val_ratio", type=float, default=0.2, help="Validation split ratio.")
    parser.add_argument("--noise", type=float, default=0.2, help="Stddev of Gaussian noise added to class logits.")

    # Model & training
    parser.add_argument("--hidden1", type=int, default=128, help="Units in first hidden layer.")
    parser.add_argument("--hidden2", type=int, default=128, help="Units in second hidden layer.")
    parser.add_argument("--activation", type=str, default="relu", help="Activation for hidden layers.")
    parser.add_argument("--dropout", type=float, default=0.0, help="Dropout rate (0 disables).")
    parser.add_argument("--learning_rate", type=float, default=1e-3, help="Adam learning rate.")
    parser.add_argument("--epochs", type=int, default=50, help="Max epochs.")
    parser.add_argument("--batch_size", type=int, default=32, help="Batch size.")
    parser.add_argument("--patience", type=int, default=10, help="Early stopping patience (epochs).")

    # Run control
    parser.add_argument("--random_state", type=int, default=42, help="Random seed.")
    parser.add_argument("--outdir", type=str, default="outputs_multiverse", help="Directory for artifacts.")
    parser.add_argument("--save_model", type=str, default="true", help="Save best/final models (true/false).")

    # Inference
    parser.add_argument(
        "--test_input",
        type=str,
        default=None,
        help="Comma-separated feature values for a single prediction, e.g., '0.8,0.1,0.3,0.9,0.2'. Must match input_dim.",
    )

    # Important for notebooks: ignore unknown args (e.g., -f <kernel.json>)
    args, _ = parser.parse_known_args()
    args.save_model = str2bool(args.save_model)
    return args


# -----------------------
# Main
# -----------------------

def main():
    args = parse_args()
    set_seed(args.random_state)

    # Data
    X, y = generate_multiverse_data(
        samples=args.samples,
        input_dim=args.input_dim,
        num_classes=args.num_classes,
        noise=args.noise,
        random_state=args.random_state,
    )
    train_ds, val_ds, (X_train, y_train, X_val, y_val) = make_datasets(
        X, y, val_ratio=args.val_ratio, batch_size=args.batch_size, seed=args.random_state
    )

    # Model
    model = build_model(
        input_dim=args.input_dim,
        num_classes=args.num_classes,
        hidden1=args.hidden1,
        hidden2=args.hidden2,
        activation=args.activation,
        dropout=args.dropout,
        learning_rate=args.learning_rate,
    )
    # Build and show summary
    _ = model(tf.zeros((1, args.input_dim), dtype=tf.float32))  # build weights
    model.summary()

    # Callbacks
    callbacks = [
        EarlyStopping(monitor="val_loss", patience=args.patience, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=max(3, args.patience // 2), min_lr=1e-6, verbose=1),
    ]
    if args.save_model:
        ensure_outdir(args.outdir)
        callbacks.append(
            ModelCheckpoint(
                filepath=os.path.join(args.outdir, "best.keras"),
                monitor="val_loss",
                save_best_only=True,
                save_weights_only=False,
                verbose=1,
            )
        )

    # Train
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=args.epochs,
        callbacks=callbacks,
        verbose=1,
    )

    # Evaluate
    val_loss, val_acc = model.evaluate(val_ds, verbose=0)
    print(f"Validation — loss: {val_loss:.6f}, accuracy: {val_acc:.4f}")

    # Optional single prediction
    test_pred = None
    if args.test_input:
        try:
            feats = np.array([float(x) for x in args.test_input.split(",")], dtype=np.float32)
            assert feats.shape[0] == args.input_dim, f"test_input length {feats.shape[0]} != input_dim {args.input_dim}"
            probs = model.predict(feats.reshape(1, -1), verbose=0)[0]
            pred_class = int(np.argmax(probs))
            test_pred = {
                "input": feats.tolist(),
                "predicted_class": pred_class,
                "probabilities": [float(p) for p in probs.tolist()],
                "top5": sorted([(int(i), float(p)) for i, p in enumerate(probs)], key=lambda t: t[1], reverse=True)[:5],
            }
            print(f"Test input -> class: {pred_class}, probs (first 5): {test_pred['probabilities'][:5]}")
        except Exception as e:
            print(f"Failed to run test prediction: {e}")

    # Save artifacts
    config = {
        "samples": args.samples,
        "input_dim": args.input_dim,
        "num_classes": args.num_classes,
        "val_ratio": args.val_ratio,
        "noise": args.noise,
        "hidden1": args.hidden1,
        "hidden2": args.hidden2,
        "activation": args.activation,
        "dropout": args.dropout,
        "learning_rate": args.learning_rate,
        "epochs": args.epochs,
        "batch_size": args.batch_size,
        "patience": args.patience,
        "random_state": args.random_state,
        "outdir": args.outdir,
    }
    metrics = {"val_loss": float(val_loss), "val_accuracy": float(val_acc)}
    if test_pred is not None:
        metrics["test_prediction"] = test_pred

    save_artifacts(
        outdir=args.outdir,
        model=model,
        config=config,
        history=history.history,
        metrics=metrics,
        save_model_flag=args.save_model,
    )
    print(f"Artifacts saved to: {args.outdir}")
    print("Done.")


if __name__ == "__main__":
    main()